In [14]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.special import factorial
from sklearn.cluster import KMeans
import os
import time
import json
from collections import namedtuple, deque
import pandas as pd
from scipy import stats
import scipy
from operator import itemgetter, attrgetter
from scipy.special import factorial
import math

In [15]:
def getCleanTripData():
    trip = pd.read_csv('./data/loc_hash_3_trip_4_2013.csv', skipinitialspace=True, dtype={"rate_code": int, "passenger_count": int,"store_and_fwd_flag": str})
    lat_min = 40.5
    lat_max = 41
    long_min = -74.25
    long_max = -73.70

    location_p_long_filter = ((trip.pickup_longitude >= long_min) & (trip.pickup_longitude <= long_max))
    location_p_lat_filter = ((trip.pickup_latitude >= lat_min) & (trip.pickup_latitude <= lat_max))
    location_d_lat_filter = ((trip.dropoff_latitude >= lat_min) & (trip.dropoff_latitude <= lat_max))
    location_d_long_filter = ((trip.dropoff_longitude >= long_min) & (trip.dropoff_longitude <= long_max))

    filtered_trips = trip[location_p_long_filter & location_p_lat_filter & location_d_long_filter & location_d_lat_filter]

    return filtered_trips

filtered_trips = getCleanTripData()

In [16]:
def assign_cluster(trips, k):
    trips_pick = trips[['pickup_longitude','pickup_latitude']]
    #trips_drop = trips[['dropoff_longitude','dropoff_latitude']]
    k_means_pick = KMeans(n_clusters=k, init='k-means++', n_init=50, max_iter=500)
    k_means_pick.fit(trips_pick)
    clust_pick = k_means_pick.labels_
    trips['pickip_cluster'] = clust_pick.tolist()
    #trips['dropoff_cluster'] = k_means_pick.predict(trips_drop)
    return trips, k_means_pick

In [ ]:
from sklearn import metrics
filtered_trips = filtered_trips.sample(50000)
for i in range(10,30,2):
    print("Started: {}".format(i))
    start = time.time()
    train_cl, k_means = assign_cluster(filtered_trips, i)  # make it 100 when extracting features 
    centroid_pickups = pd.DataFrame(k_means.cluster_centers_, columns = ['centroid_pick_long', 'centroid_pick_lat'])
    trips_pick = filtered_trips[['pickup_longitude','pickup_latitude']]
    cluster_labels = k_means.fit_predict(trips_pick)
    cl_score = metrics.silhouette_score(trips_pick, cluster_labels)
    end = time.time()
    print("Time taken for clustering is {}. Clusters: {} . Score: {}".format(end - start, i, cl_score))
    print(centroid_pickups)

Started: 10
Time taken for clustering is 905.8933057785034. Clusters: 10 . Score: 0.3868791666085636
   centroid_pick_long  centroid_pick_lat
0          -73.990969          40.751775
1          -73.785698          40.646739
2          -73.993146          40.732491
3          -73.871187          40.769556
4          -73.954778          40.773477
5          -74.007389          40.715826
6          -73.973819          40.756932
7          -73.972596          40.696438
8          -73.958959          40.801913
9          -73.980768          40.776906
Started: 12
Time taken for clustering is 881.2630228996277. Clusters: 12 . Score: 0.41153299210539185
    centroid_pick_long  centroid_pick_lat
0           -74.001573          40.739043
1           -73.870958          40.769765
2           -73.954958          40.774144
3           -73.785698          40.646739
4           -73.980476          40.777573
5           -74.006642          40.716279
6           -73.972969          40.757713
7         